## Обычный поиск оптимальных значений гиперпараметров моделей предварительной подготовки и модели машинного обучения

In [1]:
# импортируем необходимые библиотеки, функции и классы
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split, 
                                     KFold, 
                                     ParameterGrid, 
                                     cross_val_score, 
                                     GridSearchCV, 
                                     RandomizedSearchCV)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler, 
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from category_encoders import WOEEncoder, SumEncoder
from tqdm.notebook import tqdm

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')
data.head(5)

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [3]:
# преобразовываем строковые значения в целочисленные
dct = {'No': 0, 'Yes': 1}
data['Response'] = data['Response'].replace(dct)

In [4]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [5]:
# создаем списки категориальных
# и количественных столбцов
cat_columns = X_train.select_dtypes(
    include='object').columns.tolist()
num_columns = X_train.select_dtypes(
    exclude='object').columns.tolist()

In [6]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imp', SimpleImputer()),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [7]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns), 
                ('cat', cat_pipe, cat_columns)]

In [8]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [9]:
# задаем итоговый конвейер
ml_pipe = Pipeline([
    ('tf', transformer), 
    ('logreg', LogisticRegression(solver='lbfgs', 
                                  max_iter=200))
])

In [10]:
# задаем сетку гиперпараметров
param_grid = {
    'tf__num__imp__strategy': ['mean', 'median', 'constant'],    
    'tf__cat__imp__strategy': ['most_frequent', 'constant'],
    'logreg__C': [.01, .1, .5, 1, 5, 10, 100]
}

# создаем экземпляр класса GridSearchCV, передав 
# конвейер, сетку гиперпараметров и указав 
# количество блоков перекрестной проверки
gs = GridSearchCV(ml_pipe, 
                  param_grid, 
                  cv=5)
# выполняем поиск по всем значениям сетки
gs.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs.best_params_))
# смотрим наилучшее значение правильности
print("Наилучшее значение правильности: {:.3f}".format(
    gs.best_score_))
# смотрим значение правильности
# на тестовой выборке
print("Значение правильности на тестовой выборке: {:.3f}".format(
    gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров:
{'logreg__C': 0.01, 'tf__cat__imp__strategy': 'most_frequent', 'tf__num__imp__strategy': 'mean'}
Наилучшее значение правильности: 0.900
Значение правильности на тестовой выборке: 0.900


In [11]:
# извлекаем дамми-переменные, созданные классом OneHotEncoder
cat = gs.best_estimator_['tf'].named_transformers_['cat']
onehot_columns = list(cat.named_steps['ohe'].get_feature_names_out(
    input_features=cat_columns))

In [12]:
# добавляем к списку количественных переменных дамми-переменные,
# созданные OneHotEncoder, т.е. сохраняем тот же порядок
# столбцов, что задал ColumnTransformer
all_cols = num_columns + onehot_columns

In [13]:
# извлекаем константу
intercept = np.round(gs.best_estimator_['logreg'].intercept_[0], 3)
intercept

-1.978

In [14]:
# извлекаем коэффициенты
coef = np.round(gs.best_estimator_['logreg'].coef_, 3)
coef

array([[ 0.011,  0.014,  0.094, -0.038, -0.021, -0.026, -0.064, -0.041,
         0.079, -0.038, -0.058,  0.039,  0.083, -0.074,  0.01 ,  0.053,
        -0.215,  0.027,  0.53 , -0.395, -0.007,  0.007]])

In [15]:
# печатаем название "Константа"    
print("Константа:", intercept)
# печатаем название "Регрессионные коэффициенты"
print("Регрессионные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий коэффициент
for c, feature in zip(coef[0], all_cols):
    print(feature, c)

Константа: -1.978
Регрессионные коэффициенты:
Customer Lifetime Value 0.011
Income 0.014
Monthly Premium Auto 0.094
Months Since Last Claim -0.038
Months Since Policy Inception -0.021
Number of Open Complaints -0.026
Number of Policies -0.064
Coverage_Basic -0.041
Coverage_Extended 0.079
Coverage_Premium -0.038
Education_Bachelor -0.058
Education_College 0.039
Education_Doctor 0.083
Education_High School or Below -0.074
Education_Master 0.01
EmploymentStatus_Disabled 0.053
EmploymentStatus_Employed -0.215
EmploymentStatus_Medical Leave 0.027
EmploymentStatus_Retired 0.53
EmploymentStatus_Unemployed -0.395
Gender_F -0.007
Gender_M 0.007


In [16]:
# запишем результаты поиска в DataFrame
results = pd.DataFrame(gs.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(
    values=['mean_test_score'],    
    index=['param_logreg__C', 
           'param_tf__cat__imp__strategy',
           'param_tf__num__imp__strategy'])
print(table)

                                                                           mean_test_score
param_logreg__C param_tf__cat__imp__strategy param_tf__num__imp__strategy                 
0.01            constant                     constant                             0.899742
                                             mean                                 0.899742
                                             median                               0.899742
                most_frequent                constant                             0.899742
                                             mean                                 0.899742
                                             median                               0.899742
0.10            constant                     constant                             0.899569
                                             mean                                 0.899569
                                             median                               0.899569

## Обычный поиск оптимальных значений гиперпараметров моделей предварительной подготовки и модели машинного обучения с добавлением строки прогресса

In [17]:
# создаем пустой список scores, в который будем записывать
# гиперпараметры и полученные метрики качества
scores = []

# задаем метрику качества
scoring = 'accuracy'

# печатаем строку статуса
print("Выполняем поиск оптимальных значений гиперпараметров...")

# поскольку tqdm работает с итерируемыми объектами, 
# то используем класс ParameterGrid для того, чтобы 
# в явном виде перебирать гипермараметры внутри цикла for
for param in tqdm(list(ParameterGrid(param_grid)), 
                  desc='Выполнено'):
    # задаем гиперпараметры конвейера
    ml_pipe.set_params(**param)
    
    # обучаем конвейер с этими гиперпараметрами 
    # и сохраняем гиперпараметры и полученные 
    # метрики качества в список scores
    scores.append([param, cross_val_score(ml_pipe,
                                          X_train,
                                          y_train,
                                          scoring=scoring,
                                          cv=5)])
    
    # рассчитываем и добавляем в список scores 
    # усредненную метрику качества
    scores[-1].append(sum(scores[-1][1]) / len(scores[-1][1]))

# сортируем список с полученными результатами
scores.sort(reverse=True, key=lambda x: x[2])

# сохраняем и печатаем наилучшие значения гиперпараметров
best_params = scores[0][0]
print("Наилучшие значения гиперпараметров:", 
      best_params, sep='\n', end='\n')

# сохраняем и печатаем наилучшее значение метрики качества
best_score = scores[0][2]
print("Наилучшее значение %s: %.3f" % (scoring, best_score))

# передаем наилучшие значения гиперпараметров 
# в конвейер и обучаем его с этими значениями
# гиперпараметров на всей обучающей выборке
ml_pipe.set_params(**best_params).fit(X_train, y_train)
# вычисляем правильность для тестовой выборки
test_score = ml_pipe.score(X_test, y_test)
# печатаем значение метрики качества на тестовой выборке
print("Значение %s на тестовой выборке: %.3f" % (scoring, test_score))

Выполняем поиск оптимальных значений гиперпараметров...


Выполнено:   0%|          | 0/42 [00:00<?, ?it/s]

Наилучшие значения гиперпараметров:
{'logreg__C': 0.01, 'tf__cat__imp__strategy': 'most_frequent', 'tf__num__imp__strategy': 'mean'}
Наилучшее значение accuracy: 0.900
Значение accuracy на тестовой выборке: 0.900


## Случайный поиск оптимальных значений гиперпараметров моделей предварительной подготовки и модели машинного обучения

In [18]:
# создаем экземпляр класса RandomizedSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество блоков перекрестной 
# проверки, количество отбираемых значений гиперпараметров, 
# задав стартовое значение генератора псевдослучайных чисел для 
# воспроизводимости результатов
rs = RandomizedSearchCV(ml_pipe, 
                        param_grid,
                        n_iter=10,
                        cv=5,
                        random_state=42,
                        return_train_score=False)
# выполняем поиск по случайно 
# отобранным значениям из сетки
rs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    rs.best_params_))
# смотрим наилучшее значение правильности
print("Наилучшее значение правильности: {:.3f}".format(
    rs.best_score_))
# смотрим значение правильности
# на тестовой выборке
print("Значение правильности на тестовой выборке: {:.3f}".format(
    rs.score(X_test, y_test)))                                        

Наилучшие значения гиперпараметров:
{'tf__num__imp__strategy': 'median', 'tf__cat__imp__strategy': 'constant', 'logreg__C': 0.01}
Наилучшее значение правильности: 0.900
Значение правильности на тестовой выборке: 0.900


In [19]:
# запишем результаты поиска в DataFrame
res = pd.DataFrame(rs.cv_results_)
# превращаем в сводную таблицу
tbl = res.pivot_table(
    values=['mean_test_score'],    
    index=['param_logreg__C', 
           'param_tf__cat__imp__strategy',
           'param_tf__num__imp__strategy'])
print(tbl)

                                                                           mean_test_score
param_logreg__C param_tf__cat__imp__strategy param_tf__num__imp__strategy                 
0.01            constant                     median                               0.899742
0.10            most_frequent                constant                             0.899569
                                             mean                                 0.899569
0.50            most_frequent                median                               0.898363
1.00            most_frequent                median                               0.898363
5.00            constant                     constant                             0.897847
                most_frequent                constant                             0.897847
                                             median                               0.897502
10.00           most_frequent                mean                                 0.897674

## Обычный поиск оптимальных значений гиперпараметров для CatBoost при обработке категориальных признаков "как есть" ( заданы индексы категориальных признаков)

In [20]:
# создаем конвейер для категориальных переменных,
# который будем использовать с catboost
catbst_cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent'))
])

# создаем конвейер для количественных переменных,
# который будем использовать с catboost
catbst_num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median'))
])

In [21]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
catbst_transformers = [('cat', catbst_cat_pipe, cat_columns),
                       ('num', catbst_num_pipe, num_columns)]

In [22]:
# передаем список трансформеров в ColumnTransformer
catbst_transformer = ColumnTransformer(
    transformers=catbst_transformers)
# взглянем на объект
catbst_transformer

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imp',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Coverage', 'Education', 'EmploymentStatus',
                                  'Gender']),
                                ('num',
                                 Pipeline(steps=[('imp',
                                                  SimpleImputer(strategy='median'))]),
                                 ['Customer Lifetime Value', 'Income',
                                  'Monthly Premium Auto',
                                  'Months Since Last Claim',
                                  'Months Since Policy Inception',
                                  'Number of Open Complaints',
                                  'Number of Policies'])])

In [23]:
# взглянем на исходный список столбцов
print(data.columns.tolist())

['Customer Lifetime Value', 'Coverage', 'Education', 'EmploymentStatus', 'Gender', 'Income', 'Monthly Premium Auto', 'Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies', 'Response']


In [24]:
# записываем индексы категориальных признаков
# в пространстве трансформированных признаков
cat_feat_ind = [col for col in range(len(cat_columns))]
cat_feat_ind

[0, 1, 2, 3]

In [25]:
# создаем экземпляр класса CatBoostClassifier
catbst = CatBoostClassifier(n_estimators=200,
                            logging_level='Silent',
                            random_state=42,
                            cat_features=cat_feat_ind)

# задаем итоговый конвейер
catbst_pipe = Pipeline([('tf', catbst_transformer), 
                        ('catbst', catbst)])

In [26]:
# задаем сетку гиперпараметров
catbst_param_grid = {
    'tf__cat__imp__strategy': ['most_frequent', 'constant'],
    'catbst__max_depth': [4, 6, 8]
}

# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки
catbst_gs = GridSearchCV(catbst_pipe, 
                         catbst_param_grid, 
                         cv=5)
# выполняем поиск по всем значениям сетки
catbst_gs.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    catbst_gs.best_params_))
# смотрим наилучшее значение правильности
print("Наилучшее значение правильности: {:.3f}".format(
    catbst_gs.best_score_))
# смотрим значение правильности на тестовой выборке
print("Значение правильности на тестовой выборке: {:.3f}".format(
    catbst_gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров:
{'catbst__max_depth': 8, 'tf__cat__imp__strategy': 'constant'}
Наилучшее значение правильности: 0.918
Значение правильности на тестовой выборке: 0.924


In [27]:
# запишем результаты поиска в DataFrame
results = pd.DataFrame(catbst_gs.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(
    values=['mean_test_score'],    
    index=['param_catbst__max_depth', 
           'param_tf__cat__imp__strategy'])
# печатаем таблицу
print(table)

                                                      mean_test_score
param_catbst__max_depth param_tf__cat__imp__strategy                 
4                       constant                             0.903015
                        most_frequent                        0.903359
6                       constant                             0.908183
                        most_frequent                        0.907666
8                       constant                             0.917657
                        most_frequent                        0.916968


## Отбор оптимальной модели предварительной подготовки данных в рамках отдельного трансформера

In [28]:
# создаем конвейер для количественных переменных
num_pipe2 = Pipeline([
    ('imp', SimpleImputer(strategy='mean'))
])

# создаем конвейер для категориальных переменных
cat_pipe2 = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('woe', WOEEncoder(return_df=False)),
    ('sum', SumEncoder(return_df=False)),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers2 = [('num2', num_pipe2, num_columns),
                 ('cat2', cat_pipe2, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer2 = ColumnTransformer(transformers=transformers2)

# задаем итоговый конвейер
ml_pipe2 = Pipeline([
    ('tf2', transformer2), 
    ('boost', GradientBoostingClassifier(random_state=42))
])

In [29]:
# задаем сетку гиперпараметров
param_grid2 = [
    {'boost__max_depth': [4, 6, 8],
     'tf2__cat2__ohe': [None],
     'tf2__cat2__sum': [None]},
    {'boost__max_depth': [4, 6, 8],
     'tf2__cat2__woe': [None],
     'tf2__cat2__sum': [None]},
    {'boost__max_depth': [4, 6, 8],
     'tf2__cat2__woe': [None],
     'tf2__cat2__ohe': [None]}
]

In [30]:
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров, оптимизируемую метрику, указав
# стратегию перекрестной проверки
gs2 = GridSearchCV(ml_pipe2, 
                   param_grid2, 
                   cv=5)
# выполняем поиск по всем значениям сетки
gs2.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs2.best_params_))
# смотрим наилучшее значение правильности
print("Наилучшее значение правильности: {:.3f}".format(
    gs2.best_score_))
# смотрим значение правильности
# на тестовой выборке
print("Значение правильности на тестовой выборке: {:.3f}".format(
    gs2.score(X_test, y_test)))

/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/ba

/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/artemgruzdev/opt/anaconda3/lib/python3.9/site-packages/category_encoders/ba

Наилучшие значения гиперпараметров:
{'boost__max_depth': 8, 'tf2__cat2__ohe': None, 'tf2__cat2__sum': None}
Наилучшее значение правильности: 0.932
Значение правильности на тестовой выборке: 0.939


In [31]:
# увеличиваем ширину столбцов
pd.set_option('max_colwidth', 100)
# записываем результаты перекрестной 
# проверки в DataFrame
results2 = pd.DataFrame(gs2.cv_results_)
# отбираем нужные столбцы
table2 = results2.loc[:, ['param_boost__max_depth', 
                          'params', 
                          'mean_test_score']]
# переименуем столбец для удобства интерпретации
table2.rename(columns={'params': 'param_boost__encoding'}, 
              inplace=True)
# сортируем по убыванию
table2.sort_values('mean_test_score', 
                   ascending=False, 
                   inplace=True)
table2

,param_boost__max_depth,param_boost__encoding,mean_test_score
2,8,"{'boost__max_depth': 8, 'tf2__cat2__ohe': None, 'tf2__cat2__sum': None}",0.932300
8,8,"{'boost__max_depth': 8, 'tf2__cat2__ohe': None, 'tf2__cat2__woe': None}",0.929371
5,8,"{'boost__max_depth': 8, 'tf2__cat2__sum': None, 'tf2__cat2__woe': None}",0.929199
1,6,"{'boost__max_depth': 6, 'tf2__cat2__ohe': None, 'tf2__cat2__sum': None}",0.921792
7,6,"{'boost__max_depth': 6, 'tf2__cat2__ohe': None, 'tf2__cat2__woe': None}",0.919552
4,6,"{'boost__max_depth': 6, 'tf2__cat2__sum': None, 'tf2__cat2__woe': None}",0.918691
3,4,"{'boost__max_depth': 4, 'tf2__cat2__sum': None, 'tf2__cat2__woe': None}",0.906115
6,4,"{'boost__max_depth': 4, 'tf2__cat2__ohe': None, 'tf2__cat2__woe': None}",0.905943
0,4,"{'boost__max_depth': 4, 'tf2__cat2__ohe': None, 'tf2__cat2__sum': None}",0.905599


In [32]:
# создаем серию со значением кодировок, не забывая передать 
# индекс датафрейма для верного сопоставления
enc = pd.Series(['woe', 'ohe', 'sum',
                 'woe', 'sum', 'ohe',
                 'ohe', 'sum', 'woe'],
                 index = table2.index)
# создаем копию таблицы
table2_copy = table2.copy()
# значения серии становятся значениями 
# переименованного столбца
table2_copy['param_boost__encoding'] = enc
table2_copy

,param_boost__max_depth,param_boost__encoding,mean_test_score
2,8,woe,0.932300
8,8,ohe,0.929371
5,8,sum,0.929199
1,6,woe,0.921792
7,6,sum,0.919552
4,6,ohe,0.918691
3,4,ohe,0.906115
6,4,sum,0.905943
0,4,woe,0.905599


In [33]:
# присваиваем кодировки автоматически в зависимости от того, 
# какие логические условия у нас выполняются
cond = (table2['param_boost__encoding'].str.contains(
    'tf2__cat2__ohe', regex=False)) & (
    table2['param_boost__encoding'].str.contains(
        'tf2__cat2__sum', regex=False))
cond2 = (table2['param_boost__encoding'].str.contains(
    'tf2__cat2__ohe', regex=False)) & (
    table2['param_boost__encoding'].str.contains(
        'tf2__cat2__woe', regex=False))
cond3 = (table2['param_boost__encoding'].str.contains(
    'tf2__cat2__woe', regex=False)) & (
    table2['param_boost__encoding'].str.contains(
        'tf2__cat2__sum', regex=False))

table2['param_boost__encoding'] = np.where(
    cond, 'woe', (np.where(cond2, 'sum', 'ohe')))
table2

,param_boost__max_depth,param_boost__encoding,mean_test_score
2,8,woe,0.932300
8,8,sum,0.929371
5,8,ohe,0.929199
1,6,woe,0.921792
7,6,sum,0.919552
4,6,ohe,0.918691
3,4,ohe,0.906115
6,4,sum,0.905943
0,4,woe,0.905599


## Отбор оптимального метода машинного обучения среди разных методов машинного обучения (перебор значений гиперпараметров  с отдельной предобработкой данных под каждый метод машинного обучения)

In [34]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Response.csv', sep=';')
data.head(5)

,mortgage,life_ins,cre_card,deb_card,mob_bank,curr_acc,internet,perloan,savings,atm_user,markpl,age,cus_leng,response
0,No,No,No,No,No,No,No,No,No,No,No,18.0,less than 3 years,No
1,Yes,Yes,NaN,NaN,Yes,No,NaN,NaN,NaN,Yes,No,18.0,NaN,Yes
2,Yes,Yes,NaN,Yes,No,No,No,No,No,No,Yes,NaN,from 3 to 7 years,Yes
3,Yes,Yes,Yes,Yes,NaN,Yes,No,No,No,NaN,Yes,18.0,from 3 to 7 years,Yes
4,Yes,Yes,No,Yes,No,No,No,Yes,No,Yes,No,NaN,NaN,No


In [35]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
tr, tst, y_tr, y_tst = train_test_split(
    data.drop('response', axis=1), 
    data['response'], 
    test_size=.3, 
    stratify=data['response'], 
    random_state=100)

In [36]:
# создаем списки категориальных
# и количественных столбцов
categorical_features = tr.select_dtypes(
    include='object').columns.tolist()
numeric_features = tr.select_dtypes(
    exclude='object').columns.tolist()

# создаем трансформеры
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(sparse=False, 
                             handle_unknown='ignore'))
])

# передаем список трансформеров в ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# формируем итоговый конвейер
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      max_iter=400))
])  

In [37]:
# задаем сетку гиперпараметров
param_grid = [
    {'classifier': [GradientBoostingClassifier(
        n_estimators=50,
        random_state=42,
        subsample=0.8)],
     'classifier__max_depth': [4, 6, 8],
     'preprocessor__num__scaler': [None]},
    {'classifier': [LogisticRegression(solver='lbfgs', 
                                       max_iter=400)],
     'classifier__C': [.05, .01], 
     'preprocessor__num__scaler': [None]},
    {'classifier': [LogisticRegression(solver='lbfgs', 
                                       max_iter=400)],
     'classifier__C': [.05, .01]}]

In [38]:
# создаем экземпляр класса KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров, оптимизируемую метрику, указав
# стратегию перекрестной проверки
gs = GridSearchCV(pipe, 
                  param_grid, 
                  scoring='roc_auc', 
                  cv=kf)
# выполняем поиск по всем значениям сетки
gs.fit(tr, y_tr)
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs.best_params_))
# смотрим наилучшее значение AUC
print("Наилучшее значение AUC-ROC: {:.3f}".format(
    gs.best_score_))
# смотрим значение AUC на тестовой выборке
print("AUC-ROC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_tst, gs.predict_proba(tst)[:, 1])))

Наилучшие значения гиперпараметров:
{'classifier': GradientBoostingClassifier(max_depth=4, n_estimators=50, random_state=42,
                           subsample=0.8), 'classifier__max_depth': 4, 'preprocessor__num__scaler': None}
Наилучшее значение AUC-ROC: 0.910
AUC-ROC на тестовой выборке: 0.907


In [39]:
# создаем трансформеры для логистической регрессии
numeric_transformer_logreg = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_logreg = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(sparse=False, 
                             handle_unknown='ignore'))
])

# передаем список трансформеров для логистической 
# регрессии в ColumnTransformer
preprocessor_logreg = ColumnTransformer([
    ('num', numeric_transformer_logreg, numeric_features),
    ('cat', categorical_transformer_logreg, categorical_features)
])

# формируем итоговый конвейер для логистической регрессии
pipe_logreg = Pipeline([
    ('preprocessor', preprocessor_logreg),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      max_iter=400))
])

In [40]:
# создаем трансформеры для градиентного бустинга
numeric_transformer_boost = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=-9999))
])

categorical_transformer_boost = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('sum', SumEncoder(return_df=False))
])

# передаем список трансформеров для градиентного 
# бустинга в ColumnTransformer
preprocessor_boost = ColumnTransformer([
    ('num', numeric_transformer_boost, numeric_features),
    ('cat', categorical_transformer_boost, categorical_features)
])

# формируем итоговый конвейер для градиентного бустинга
pipe_boost = Pipeline([
    ('preprocessor', preprocessor_boost),
    ('classifier', GradientBoostingClassifier(
        random_state=42, subsample=0.8))
])  

In [41]:
# задаем сетки значений гиперпараметров
param_logreg_scaled = [{'classifier__C': [.05, .01]}] 
param_logreg_non_scaled = [{'classifier__C': [.05, .01], 
                            'preprocessor__num__scaler': [None]}] 
param_boost = [{'classifier__max_depth': [2, 4], 
                'classifier__n_estimators': [50, 100]}]

In [42]:
# создаем экземпляры класса GridSearchCV
gs_logreg_scaled = GridSearchCV(
    pipe_logreg, 
    param_logreg_scaled, 
    scoring='roc_auc', 
    cv=5)

gs_boost = GridSearchCV(
    pipe_boost, 
    param_boost, 
    scoring='roc_auc', 
    cv=5)

gs_logreg_non_scaled = GridSearchCV(
    pipe_logreg,
    param_logreg_non_scaled, 
    scoring='roc_auc', 
    cv=5)

# объединяем в список
grids = [gs_logreg_scaled, gs_boost, gs_logreg_non_scaled]

In [43]:
# создаем словарь для сопоставления индекса
# с названием метода машинного обучения
grid_dict = {0: 'Логистическая регрессия со стандартизацией', 
             1: 'Градиентный бустинг',
             2: 'Логистическая регрессия без стандартизации'} 

In [44]:
# отключим предупреждения касательно поведения в будущем
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# печатаем строку статуса
print("Выполняем поиск оптимальных значений гиперпараметров...")
# добавляем пустую строку в вывод
print("")
# создаем пустой список, в который будем записывать наилучшее 
# значение AUC по каждому методу машинного обучения
auc_list = []
# здесь будем хранить индекс, соответствующий 
# методу машинного обучения
best_clf = 0
# выполняем поиск по сетке
for idx, gs in enumerate(grids):
    # печатаем метод машинного обучения
    print("Метод: %s" % grid_dict[idx])
    gs.fit(tr, y_tr)
    # печатаем наилучшие значения гиперпараметров для этого метода
    print("Наилучшие значения гиперпараметров: %s" % gs.best_params_)
    # печатаем наилучшее значение AUC для этого метода
    print("Наилучшее значение AUC-ROC: %.3f" % gs.best_score_)
    # добавляем пустую строку в вывод
    print("")
    # записываем наилучшее значение AUC для метода машинного обучения
    auc_score = gs.best_score_
    # добавляем наилучшее значение AUC в список
    auc_list.append(auc_score)
    # если мы получаем максимальное значение AUC
    if auc_score == max(auc_list):
        # записываем индекс наилучшего метода
        best_clf = idx
        # записываем значения гиперпараметров наилучшего метода
        bst_params = gs.best_params_
        # вычисляем вероятности положительного класса
        proba_tst = gs.predict_proba(tst)[:, 1]
        # вычисляем AUC на тестовой выборке
        auc_tst = roc_auc_score(y_tst, proba_tst)
# добавляем пустую строку в вывод
print("")
# печатаем название лучшего метода, используем словарь
# для сопоставления индекса с названием  метода
# машинного обучения
print("Лучший метод машинного обучения: %s" % grid_dict[best_clf])
# печатаем значения гиперпараметров лучшего метода
print("Значения гиперпараметров лучшего метода: %s" % bst_params)
# печатаем AUC лучшего метода на тестовой выборке
print("AUC-ROC лучшего метода на тестовой выборке: %.3f" % auc_tst)

Выполняем поиск оптимальных значений гиперпараметров...

Метод: Логистическая регрессия со стандартизацией
Наилучшие значения гиперпараметров: {'classifier__C': 0.05}
Наилучшее значение AUC-ROC: 0.905

Метод: Градиентный бустинг
Наилучшие значения гиперпараметров: {'classifier__max_depth': 4, 'classifier__n_estimators': 50}
Наилучшее значение AUC-ROC: 0.909

Метод: Логистическая регрессия без стандартизации
Наилучшие значения гиперпараметров: {'classifier__C': 0.05, 'preprocessor__num__scaler': None}
Наилучшее значение AUC-ROC: 0.905


Лучший метод машинного обучения: Градиентный бустинг
Значения гиперпараметров лучшего метода: {'classifier__max_depth': 4, 'classifier__n_estimators': 50}
AUC-ROC лучшего метода на тестовой выборке: 0.907
